# ARC Tools
## TS Guesses
Enter smiles of reactants and products. Write ``None`` if needed.

In [ ]:
from arc.imports import settings
from arc.job.adapters.common import all_families_ts_adapters, ts_adapters_by_rmg_family
from arc.plotter import draw_structure
from arc.reaction import ARCReaction
from arc.rmgdb import make_rmg_database_object, load_families_only
from arc.species import ARCSpecies
from arc.species.converter import xyz_to_str
from arc.scheduler import Scheduler

%matplotlib inline

In [ ]:
r1_smiles = '[OH]'
r2_smiles = 'CCC'

p1_smiles = 'O'
p2_smiles = '[CH2]CC'

dihedral_increment = 60

In [ ]:
rmgdb = make_rmg_database_object()
load_families_only(rmgdb)

In [ ]:
def generate_species_from_smiles(smiles):
    return ARCSpecies(label=smiles, smiles=smiles)

In [ ]:
r_species, p_species = list(), list()
for r_smiles in [r1_smiles, r2_smiles]:
    if r_smiles is not None:
        r_species.append(generate_species_from_smiles(r_smiles))
for p_smiles in [p1_smiles, p2_smiles]:
    if p_smiles is not None:
        p_species.append(generate_species_from_smiles(p_smiles))

rxn = ARCReaction(r_species=r_species, p_species=p_species)
rxn.ts_species = ARCSpecies(label='TS', is_ts=True)
rxn.determine_family(rmg_database=rmgdb)

print(f'Considering the following reaction:\n\n{rxn.label}\n\nReaction family: {rxn.family.label}')

In [ ]:
sched = Scheduler(project='TS_search_1',
                  species_list=rxn.r_species + rxn.p_species,
                  rmg_database=rmgdb,
                  conformer_level='wb97xd/Def2TZVP',
                  opt_level='wb97xd/Def2TZVP',
                  sp_level='wb97xd/Def2TZVP',
                  freq_level='wb97xd/Def2TZVP',
                  ess_settings=None,
                  testing=True,
                  project_directory='~/Code/TS_search_1',
                 )
sched.job_dict = {'TS': {'tsg': {}}}

In [ ]:
tsg_index = 0
#ts_adapters = settings['ts_adapters']
ts_adapters = ['heuristics', 'xtb_gsm']
ts_adapters = [a.lower() for a in ts_adapters]

print(f'Considering the following TS Search methods: {ts_adapters}')

for method in ts_adapters:
    if method in all_families_ts_adapters or \
            (rxn.family is not None
             and rxn.family.label in list(ts_adapters_by_rmg_family.keys())
             and method in ts_adapters_by_rmg_family[rxn.family.label]):
        sched.run_job(job_type='tsg',
                      job_adapter=method,
                      reactions=[rxn],
                      tsg=tsg_index,
                      dihedral_increment=dihedral_increment,
                     )
        tsg_index += 1

print(f'Got {len(rxn.ts_species.ts_guesses)} TS Guesses (before clustering).')

rxn.ts_species.cluster_tsgs()
        
print(f'Got {len(rxn.ts_species.ts_guesses)} TS Guesses (after clustering).')

In [ ]:
for tsg in rxn.ts_species.ts_guesses:
    print(f'{tsg.method} {tsg.method_index} (success: {tsg.success}):')
    print(f'coordinates:\n{xyz_to_str(tsg.initial_xyz)}')
    if tsg.initial_xyz is not None:
        draw_structure(xyz=tsg.initial_xyz)
    print('\n\n\n')